# Project Group 18

Members: Karin van den Berg, Wouter Diebels, Floris Muis, Levi Mulder, Maaike Tjeerdsma

Student numbers: 4938933, 5869323, 5110394, 4712463, 4964578

# Research Objective

## Introduction
Nearly 60% of the Netherlands is flood prone [Ligtvoet, 2009]. 55% of this area includes the inland area that is protected by dunes and dikes [Minnen et al., 2012]. Regarding the floods, climate change influences the safety of the Netherlands in different ways. Besides the rise of the sea level, one of the most important factors that increase the risk of floods are the increasing peak discharges of rivers [Minnen et al., 2012]. For instance, in 2021, the extreme weather conditions and the substantial waterlevels of the rivers cause the second most expensive nature disaster of that particular year [nos, 2021]. In contrast to the high-waterlevels, climate change also causes drought. Due to the low waterlevels, freight transport via inland shipping has been difficult for some time. The waterlevel in the Rhine dropped to such an extent that shipping is hampered [Parool, 2022]. The ships can carry less so as not to lie too deep which causes more pressure on the freight transport via inland shipping. If climate change more often leads to extreme waterlevels and river discharges, the risk of these incidents will rise [Klijn et al., 2010]. According to [Baede, 2001], climate change refers to the average weather in terms of the mean and its variability over a certain timespan and a certain area [Baede, 2001]. The research objective of the project is therefore to investigate the extent to which climate change, in other words changes in temperature, affect Dutch waterlevels. Our hypothesis is that the changes in temperature play a major role in influencing the waterlevels in the Netherlands. The expectation is that the rise in average temperature is expected ti incrase the variance of the waterlevel height.

## Research Question
To what extend does global temperature change influence Dutch river level heights?

## Method
### Data
The data for the waterlevel heights used for this assignment was retrieved from Rijkswaterstaat. Data was available from 1980 (which was left out of further analysis) and 1987 until 2022, taken at the measuring points Eijsden and Lobith. These towns were selected because they lie on the Dutch border at the places where the Maas and the Rhine enter the Netherlands. The data gives the water level height compared to the NAP, not to the bottom of the river. This however will not be an issue, since only the variances and standard deviations will be used in this research.

For the European temperatures, data from the National Centers for Environmental Information (NOAA) was used. From 1987 until August 2022, the monthly average temperature of Europe was selected. The dataset presents the anomaly of the monthly temperature in °C, relative to the average of the base period 1910-2000.
### Visualization
For the visualization, an interactive plot will be made, showing the waterlevels and the average temperature, combined with a slider for the time. 
Also, a second plot will be made, plotting time against both waterlevel heights and temperature. This plot will also show the linear regressions of the temperature and the river level heights
### Analysis
In the hypothesis it was stated that the rise in average temperature is expected to increase the variance of the waterlevel height. The variance and the standard deviation of the waterlevel height differences are measured over the time spans of days, months and years. A Pearson’s r correlation test will be done on the temperature and the change of variance of waterlevel heights.


# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Karin van den Berg**: Requesting and downloading temperature data, calculating variance

**Wouter Diebels**: Writing introduction, plotting data temperature and waterlevel height

**Floris Muis**: Cleaning waterlevel height data, implementation of comments on code

**Levi Mulder**: Cleaning temperature data, plotting data temperature and waterlevel height

**Maaike Tjeerdsma**: Writing methods, implementation of text into the project template


# Data Used

- [nos, 2021] (2021). Overstromingen in limburg en buurlanden op één na duurste natuurramp van 2021.
- [Baede, 2001] Baede, A. P. (2001). The climate system: an overview. Climate change 2001: the scientific basis, pages 38–47.
- [Klijn et al., 2010] Klijn, F., Kwadijk, J., de Bruijn, K., and Hunink, J. (2010). Overstromingsrisico’s en droogterisico’s in een veranderend klimaat: verkenning van wegen naar een klimaatveranderingsbestendig Nederland. Deltares Delft.
- [Ligtvoet, 2009] Ligtvoet, W. (2009). Roadmap for a climate-proof netherlands; wegen naar een klimaatbestendig nederland.
- [Minnen et al., 2012] Minnen, J. V., Ligtvoet, W., Bree, L. v., Hollander, G. d., Visser, H., Schrier, G., Bessembinder, J., van Oldenborgh, G., Prozny, T., Sluijter, R., et al. (2012). Effecten van klimaatverandering in nederland: 2012. Beleidsstudies, pages 1–125.
- [Parool, 2022] Parool, H. (2022). Dalend waterpeil in de Rijn geeft problemen voor goederenvervoer binnenvaart — parool.nl. https://www.parool.nl/nederland/dalend-waterpeil-in-de-rijn-geeft-problemen-voor-goederenvervoer-binnenvaart b670f116/. [Accessed 13-Oct-2022].


# Data Pipeline

## Import modules and data

In [1]:
# importing the necessary libraries
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats
import datetime as dt
import math

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# notes for group 18 members, ourselves
    # remove these files from your local github directory after using them, otherwise you will get a push notification saying: 
    # "file size exeeds 100mb, failed to push"
    # These rawdata files can be found on our google drive folder. (for group 18 members, ourselves)

# general note 
    # These rawdata files are in the folder 'Rawdata' as .zip files, you need to open them in the same folder 

filePath1 = "Rawdata/20221012_030.csv" #data from 20221012_030.zip
filePath2 = "Rawdata/20221012_030_2nd.csv" #data from 20221012_030 (1).zip, needs to be renamed to 20221012_030_2nd.csv
filePath3 = 'Rawdata_temperature/Temperature_data.csv'
filterSize = 5 #number of standard deviations for which values are not considered outliers. 

In [3]:
# activates the data_import function (this takes quite a long time due to the filesize)
rawDataEijsden =  pd.read_csv(filePath1, delimiter=";", encoding='latin-1') 
rawDataLobith =  pd.read_csv(filePath2, delimiter=";", encoding='latin-1') 
rawDataTemp = pd.read_csv(filePath3, delimiter=',', skiprows=[0,1,2,3]) #read csv and skip first 4 rows with non usable data

## Functions

In [4]:
# function to reduce the amount of data from the riverwater level heights so it is usable
def data_cleaner(data):
    selectedData = data.iloc[:,[21,22,24]]
    selectedData = selectedData.iloc[::144,:] #144 = 6 * 24 to reduce the amount of rows to 1 row per day. 
    locationName = data.iloc[1,1]
    selectedData = selectedData[(np.abs(stats.zscore(selectedData["NUMERIEKEWAARDE"])) < filterSize)] #filters out outliers
    selectedData['WAARNEMINGDATUM'] = pd.to_datetime(selectedData['WAARNEMINGDATUM'], format='%d-%m-%Y') #data from YYYYMM to DD-MM-YYYY
    return selectedData, locationName

# function to clean the raw data from the temperature so it is usable
def data_cleaner_temperature(data):
    data['Year'] = pd.to_datetime(data['Year'], format='%Y%m') #date from YYYYMM to YYYY-MM-DD
    return data  


# function to create the first visuals of the river level heights
def first_visual(data, plotTitle):
    fig = px.line(data,title=plotTitle, x="WAARNEMINGDATUM", y="NUMERIEKEWAARDE")
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=10, label="10y", step="year", stepmode="backward"), #de legenda moet nog even gefixt worden. geen idee nog hoe
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    
    fig.show()
    return


# functions variance
def variance_Eijsden(data): 
    number_data = len(data)
    deviations = [(p - mean_all_data_Eijsden)**2 for p in data]
    variance = sum(deviations) / number_data
    return variance

def variance_Lobith(data): 
    number_data = len(data)
    deviations = [(p - mean_all_data_Lobith)**2 for p in data]
    variance = sum(deviations) / number_data
    return variance


# functions standard deviation
def stddev_Eijsden(data):
    variance_data = variance_Eijsden(data)
    stddev_data = math.sqrt(variance_data)
    return stddev_data

def stddev_Lobith(data):
    variance_data = variance_Lobith(data)
    stddev_data = math.sqrt(variance_data)
    return stddev_data


# functions variance for specific years
def variance_year_Eijsden(chosen_year):
    data_Eijsden_year = dataEijsden[pd.DatetimeIndex(dataEijsden['WAARNEMINGDATUM']).year==chosen_year]
    return (variance_Eijsden(data_Eijsden_year['NUMERIEKEWAARDE']))

def variance_year_Lobith(chosen_year):
    data_Lobith_year = dataLobith[pd.DatetimeIndex(dataLobith['WAARNEMINGDATUM']).year==chosen_year]
    return (variance_Lobith(data_Lobith_year['NUMERIEKEWAARDE']))


# functions standard deviation for specific years
def stdev_year_Eijsden(chosen_year):
    data_Eijsden_year = dataEijsden[pd.DatetimeIndex(dataEijsden['WAARNEMINGDATUM']).year==chosen_year]
    return (stddev_Eijsden(data_Eijsden_year['NUMERIEKEWAARDE']))

def stdev_year_Lobith(chosen_year):
    data_Lobith_year = dataLobith[pd.DatetimeIndex(dataLobith['WAARNEMINGDATUM']).year==chosen_year]
    return (stddev_Lobith(data_Lobith_year['NUMERIEKEWAARDE']))


# function to merge the graphs from the standard deviation river levels and temperature
def figure_merge(data_merge):
    trace1 = go.Scatter(x=data_merge['WAARNEMINGDATUM'],
                    y=data_merge['NUMERIEKEWAARDE'],
                    name='Water level',
                    mode='lines+markers',
                    yaxis='y1')
    trace2 = go.Scatter(x=data_merge['WAARNEMINGDATUM'],
                    y=data_merge['Value'],
                    name='Temperature',
                    mode='lines+markers',
                    yaxis='y2')
    data = [trace1, trace2]
    layout = go.Layout(title= 'Standard deviation river levels vs Temperature',
                   yaxis=dict(title='Water level'),
                   yaxis2=dict(title='Temperature anomalies in Celsius',
                               overlaying='y',
                               side='right'))

    Figure = go.Figure(data=data, layout=layout)
    return Figure

## Visualisations

First, we'll show the river level heights measured in Lobith and Eijsden. The heights are measured compared to NAP, not the heights compared to the bottom of the river. With the slider it is made possible to take a quick look at different timeframes.

In [5]:
# retrieving the river level height data
dataEijsden, locationName1 = data_cleaner(rawDataEijsden)
dataLobith, locationName2 = data_cleaner(rawDataLobith)

# creating visuals of the river level heights
first_visual(dataEijsden,locationName1)
first_visual(dataLobith,locationName2)

Next up are the anomalies of temperature in Europe. The definition of the anomalies is discussed in the Methods section.

In [6]:
# retrieving temperature data
dataTemp = data_cleaner_temperature(rawDataTemp)

# plot europe temperature anomalies
fig = px.line(dataTemp, title = 'Europe temperature anomalies', x = 'Year', y = 'Value')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=1, label="Year-to-date", step="year", stepmode="todate"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

## Variances waterlevel heights
We'll now start with showing the variances of the waterlevel heights
### Variance per year

In [7]:
# loop through years
year_number = range(1987, 2023)
variance_list_Eijsden = []
variance_list_Lobith = []


# necessary to run the code below
mean_all_data_Eijsden = sum(dataEijsden['NUMERIEKEWAARDE'])/len(dataEijsden)
mean_all_data_Lobith = sum(dataLobith['NUMERIEKEWAARDE'])/len(dataLobith)


def variance_year_Eijsden(chosen_year):
    data_Eijsden_year = dataEijsden[pd.DatetimeIndex(dataEijsden['WAARNEMINGDATUM']).year==chosen_year]
    return (variance_Eijsden(data_Eijsden_year['NUMERIEKEWAARDE']))

def variance_year_Lobith(chosen_year):
    data_Lobith_year = dataLobith[pd.DatetimeIndex(dataLobith['WAARNEMINGDATUM']).year==chosen_year]
    return (variance_Lobith(data_Lobith_year['NUMERIEKEWAARDE']))

for n in year_number:
    numbers1 = variance_year_Eijsden(n)
    variance_list_Eijsden.append(numbers1)
    
for n in year_number:
    numbers2 = variance_year_Lobith(n)
    variance_list_Lobith.append(numbers2)

In [8]:
# plot variance Eijsden
x = year_number
y = variance_list_Eijsden
fig = px.line(dataEijsden, title = 'variance Eijsden', x = year_number , y = variance_list_Eijsden )
fig.show()

In [9]:
# plot variance Lobith
x = year_number
y = variance_list_Lobith
fig = px.line(dataLobith, title = 'variance Lobith', x = year_number, y = variance_list_Lobith)
fig.show()

### Variance per day/month
To be continued...

## Standard deviation waterlevel heights
We'll now start with showing the deviation of the waterlevel heights relative to the mean!
### Deviation from the mean per day

In [10]:
# deviation per day Eijsden
deviation_day_Eijsden = []

mean_all_data_Eijsden = sum(dataEijsden['NUMERIEKEWAARDE'])/len(dataEijsden)
for p in (dataEijsden['NUMERIEKEWAARDE']):
    deviation_day_Eijsden.append((p - mean_all_data_Eijsden))


# plot deviation Eijsden
fig = px.line(dataEijsden, title = 'Deviation Eijsden', x = 'WAARNEMINGDATUM', y = deviation_day_Eijsden)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=6, label="6m", step="month", stepmode="backward"), #de legenda moet nog even gefixt worden. geen idee nog hoe
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [11]:
# deviation per day Lobith
deviation_day_Lobith = []

mean_all_data_Lobith = sum(dataLobith['NUMERIEKEWAARDE'])/len(dataLobith)
for p in (dataLobith['NUMERIEKEWAARDE']):
    deviation_day_Lobith.append((p - mean_all_data_Lobith))


# plot deviation Lobith
fig = px.line(dataLobith, title = 'Deviation Lobith', x = 'WAARNEMINGDATUM', y = deviation_day_Lobith)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=6, label="6m", step="month", stepmode="backward"), #de legenda moet nog even gefixt worden. geen idee nog hoe
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(title='standard deviation')
fig.show()

### Standard deviation per year

In [12]:
stdev_list_Eijsden = []
stdev_list_Lobith = []

# deviation per year Eijsden
for n in year_number:
    numbers3 = stdev_year_Eijsden(n)
    stdev_list_Eijsden.append(numbers3)

# deviation per year Lobith
for n in year_number:
    numbers4 = stdev_year_Lobith(n)
    stdev_list_Lobith.append(numbers4)

In [13]:
# plot standard deviation Eijsden
fig = px.scatter(dataEijsden, title = 'Standard Deviation Eijsden', x = year_number, y = stdev_list_Eijsden, trendline = 'ols')
fig.show()

In [14]:
# plot standard deviation Lobith
fig = px.scatter(title = 'Standard Deviation Lobith', x = year_number, y = stdev_list_Lobith, trendline = 'ols')
fig.show()

### Standard deviation of water level correlation with average temperature (pearson correlation)


In [30]:
from plotly import data


def temp_averager(dataTemp, reductionNum=12):
    for i in range(len(dataTemp)):
        continue

temp_averager(dataTemp=dataTemp["Value"], reductionNum=12)

# dataTempReduced = dataTemp.rolling("12m", min_periods=1, step=12).sum()

dataTempReduced = dataTemp.groupby(np.arange(len(dataTemp))//12).mean()

dataTempReducedstdevLobith = dataTempReduced
dataTempReducedstdevLobith["stdevLobith"] = stdev_list_Lobith

fig = px.scatter(title = 'Standard Deviation vs temp per year', x = dataTempReduced["Value"], y = stdev_list_Lobith)
fig.show()


pearsonCor = dataTempReducedstdevLobith["Value"].corr(dataTempReducedstdevLobith["stdevLobith"])
print("The pearson correlation coefficient between the variance per year of the river level heights and the average temperature rise per year is:", pearsonCor)







36
range(1987, 2023)
<class 'pandas.core.frame.DataFrame'>
          Year  Value
0   1987-01-01  -3.78
1   1987-02-01  -0.01
2   1987-03-01  -2.99
3   1987-04-01  -0.25
4   1987-05-01  -0.87
..         ...    ...
423 2022-04-01   0.79
424 2022-05-01   1.40
425 2022-06-01   2.52
426 2022-07-01   1.90
427 2022-08-01   2.82

[428 rows x 2 columns]
           Value
0  -6.325000e-01
1   2.925000e-01
2   1.055833e+00
3   1.028333e+00
4   2.041667e-01
5   4.433333e-01
6   1.850372e-17
7   8.500000e-01
8   6.200000e-01
9  -1.508333e-01
10  4.758333e-01
11  6.008333e-01
12  1.076667e+00
13  1.260833e+00
14  8.708333e-01
15  1.100000e+00
16  8.958333e-01
17  7.533333e-01
18  7.791667e-01
19  1.068333e+00
20  1.379167e+00
21  1.272500e+00
22  1.067500e+00
23  5.933333e-01
24  1.218333e+00
25  9.791667e-01
26  1.044167e+00
27  1.784167e+00
28  1.712500e+00
29  1.487500e+00
30  1.390000e+00
31  1.868333e+00
32  1.827500e+00
33  2.155833e+00
34  1.257500e+00
35  1.962500e+00
           Value  stdevL

The pearson correlation coefficient between the variance per year of the river level heights and the average temperature rise per year is: -0.15052776253523537


### Merging graphs of Standard deviation river levels vs Temperature
To be able to compare the river level heights with the temperature, the graphs of the Standard deviation river levels and the Temperature are merged. On the x-axis is the corresponding timescale, on the left y-axis the water level and on the right y-axis the temperature anomalies.

In [16]:
dataTemperature_merge=dataTemp.rename({'Year': 'WAARNEMINGDATUM'}, axis=1)

df_Lobith_temp = pd.merge(dataLobith, dataTemperature_merge, how='outer', on=['WAARNEMINGDATUM'])
df_Eijsden_temp = pd.merge(dataEijsden, dataTemperature_merge, how='outer', on=['WAARNEMINGDATUM'])

# figures merge lobith and temp
figure_merge(df_Lobith_temp)

# figure merge Eijsden and temp
figure_merge(df_Eijsden_temp)